In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%aimport sys
sys.path.insert(0,'..')

In [3]:
%env TEXTMORPH_DATA=/Users/guillaume/Desktop/backup/neural-editor-grll-data

env: TEXTMORPH_DATA=/Users/guillaume/Desktop/backup/neural-editor-grll-data


In [15]:
from os.path import dirname, realpath, join

from textmorph.text_generation.training_run import EditTrainingRuns
from textmorph import data
from textmorph.text_generation.training_run import GenData
from gtd.ml.torch.utils import similar_size_batches
from gtd.chrono import verboserate

In [5]:
data_dir = join(data.workspace.root, "textgen_dataset")

In [6]:
source_data = GenData(data_dir)

[localhost] local: wc -l /Users/guillaume/Desktop/backup/neural-editor-grll-data/textgen_dataset/IVR_text_HiGe_SCinternal


In [12]:
experiments = EditTrainingRuns(check_commit=False)
exp_id = [6]
exp = experiments[int(exp_id[0])]

Reloaded checkpoint #100000


SEJveChjaGlsZHJlbj0oSW50UHJvZ3Jlc3ModmFsdWU9MCwgZGVzY3JpcHRpb249dSdMb2FkaW5nIGVtYmVkZGluZ3MgZnJvbSAvVXNlcnMvZ3VpbGxhdW1lL0Rlc2t0b3AvYmFja3VwL25ldXLigKY=



No GPUs detected. Sticking with CPUs.


In [13]:
exp.editor

Editor(
  (encoder): Encoder(
    (token_embedder): TokenEmbedder(
      (_embedding): TrainFlagEmbedding(
        (_embedding): Embedding(20003, 300)
      )
    )
    (source_encoder): MultiLayerSourceEncoder(
      (encoder_layer_0): BidirectionalSourceEncoder(
        (forward_encoder): SimpleSourceEncoder(
          (rnn_cell): LSTMCell(300, 128)
        )
        (backward_encoder): SimpleSourceEncoder(
          (rnn_cell): LSTMCell(300, 128)
        )
      )
      (encoder_layer_1): BidirectionalSourceEncoder(
        (forward_encoder): SimpleSourceEncoder(
          (rnn_cell): LSTMCell(256, 128)
        )
        (backward_encoder): SimpleSourceEncoder(
          (rnn_cell): LSTMCell(256, 128)
        )
      )
      (encoder_layer_2): BidirectionalSourceEncoder(
        (forward_encoder): SimpleSourceEncoder(
          (rnn_cell): LSTMCell(256, 128)
        )
        (backward_encoder): SimpleSourceEncoder(
          (rnn_cell): LSTMCell(256, 128)
        )
      )
    )
  

In [17]:
batches = similar_size_batches(source_data.data, 32, size=lambda x: len(x))

Optimized batches: reduced cost from 1568 (naive) to 1568 (0.0% reduction).
Optimal (batch_size=1) would be 1456.


In [34]:
for batch in verboserate(batches, desc='Streaming Source Sentences'):
    # Source Encode
    source_words, insert_words, insert_exact_words, delete_words, delete_exact_words, target_words, edit_embed = exp.editor._batch_editor_examples(batch)
    encoder_input = exp.editor.encoder.preprocess(source_words, insert_words, insert_exact_words, delete_words, delete_exact_words, edit_embed)
    encoder_output = exp.editor.encoder(encoder_input)

    break

In [35]:
encoder_output

EncoderOutput(source_embeds=SequenceBatch(values=tensor([[[ 0.0105, -0.0172,  0.0120,  ..., -0.0238,  0.0066, -0.0048],
         [-0.0010, -0.2149,  0.0760,  ..., -0.0335,  0.0214,  0.0278],
         [-0.0102,  0.3015,  0.0230,  ..., -0.0974,  0.0172,  0.0054],
         ...,
         [-0.0095, -0.0314, -0.0167,  ..., -0.1196,  0.1468, -0.1008],
         [-0.0095, -0.0314, -0.0167,  ..., -0.1196,  0.1468, -0.1008],
         [-0.0095, -0.0314, -0.0167,  ..., -0.1196,  0.1468, -0.1008]],

        [[ 0.0102, -0.0176,  0.0121,  ..., -0.0676,  0.0052, -0.0025],
         [-0.0011, -0.2028,  0.0799,  ..., -0.0741,  0.0168,  0.0328],
         [-0.0104,  0.2999,  0.0268,  ..., -0.1959,  0.0155,  0.0107],
         ...,
         [-0.1257, -0.0476,  0.4313,  ..., -0.1196,  0.1468, -0.1008],
         [-0.1257, -0.0476,  0.4313,  ..., -0.1196,  0.1468, -0.1008],
         [-0.1257, -0.0476,  0.4313,  ..., -0.1196,  0.1468, -0.1008]],

        [[ 0.0102, -0.0176,  0.0121,  ..., -0.0676,  0.0052, -0.002